# Create and test your own forge

In [1]:
import asyncio
import sys
from datetime import UTC, datetime
from pathlib import Path
import random
from typing import Literal
from pydantic import BaseModel
import nest_asyncio
nest_asyncio.apply()

# loading dotenv
from dotenv import load_dotenv
load_dotenv()

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge
from ebiose.core.evo_forging_cycle import EvoForgingCycleConfig

from loguru import logger
logger.remove()
logger.add(sys.stderr, level="DEBUG")

/Users/xabier/dev/ebiose/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


2

### 1. Describe the problem you wish to solve with an Ebiose agent

In [ ]:
PROBLEM_DESCRIPTION = """Given my CRM API connection, 
run a mailing campaign to all my customers who have not purchased anything 
in the last 6 months. The email should be personalized and include a discount 
code for their next purchase. The email should also include a link to my website 
and a call to action to encourage them to make a purchase."""

### 2. Define the I/O model of your agent

In [ ]:
class AgentInput(BaseModel):
    pass

class AgentOutput(BaseModel):
    pass

### 3. Create the forge class

In [ ]:
class YourForge(AgentForge):
    name: str = "YourForge"
    description: str = PROBLEM_DESCRIPTION

    agent_input_model: type[BaseModel] = AgentInput
    agent_output_model: type[BaseModel] = AgentOutput
    default_generated_agent_engine_type: str = "langgraph_engine"

    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:  # noqa: C901
        return random.random()

### 4. Setup a forge cycle

In [ ]:
# run parameters
BUDGET = 0.5 # budget in dollars
N_AGENTS = 2 # number of agents by generation

# the path where results will be saved
timestamp = datetime.now(tz=UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./data/{timestamp}/")
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)

# node types
NODE_TYPES = [
    "StartNode", "EndNode", "LLMNode", 
]
MOCK_NODE_TYPES = [
    "PythonNode", "DatabaseNode", "APINode", "WebScraperNode",
    "FileNode", "UserQueryNode"
] # these nodes are not implemented yet

cycle_config = EvoForgingCycleConfig(
    budget=BUDGET,
    n_agents_in_population=N_AGENTS,
    n_selected_agents_from_ecosystem=0,
    n_best_agents_to_return=2,
    replacement_ratio=0.5,
    save_path=SAVE_PATH,
    node_types= NODE_TYPES + MOCK_NODE_TYPES,

)

### 5. Run a forge cycle

In [ ]:
# instantiate the forge
forge = YourForge()

best_agents, best_finess = asyncio.run(
    forge.run_new_cycle(
        config=cycle_config
    ),
)

### 6. Display final results

In [ ]:
forge.display_results(best_agents, best_finess)

In [5]:
best_agents, best_finess = asyncio.run(
    forge.run_new_cycle(config=cycle_config),
)

2025-04-08 16:46:11.267 | INFO     | ebiose.core.evo_forging_cycle:execute_a_cycle:180 - Starting a new cycle for forge YourForge
2025-04-08 16:46:11.268 | INFO     | ebiose.core.evo_forging_cycle:initialize_population:112 - ****** Initializing agents population ******
2025-04-08 16:46:11.268 | INFO     | ebiose.core.evo_forging_cycle:initialize_population:124 - Creating 2 new agents with architect agents...
  0%|          | 0/2 [00:00<?, ?it/s]2025-04-08 16:47:11.009 | DEBUG    | ebiose.core.engines.graph_engine.utils:get_structured_output_agent:50 - 
Initializing structured output agent for model AgentOutput (1)
100%|██████████| 2/2 [03:01<00:00, 90.50s/it] 
2025-04-08 16:49:12.280 | DEBUG    | ebiose.core.evo_forging_cycle:initialize_population:169 - Agent initialization cost: 0.01176945
2025-04-08 16:49:12.280 | INFO     | ebiose.core.evo_forging_cycle:initialize_population:175 - Population initialized with 2 agents
2025-04-08 16:49:12.280 | INFO     | ebiose.core.evo_forging_cycle

# Agent ID: agent-7951d8f3-127b-4aa3-86d3-34cb1f24e51e
## Fitness: 0.8998362177442522
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(IdentifyInactiveCustomers)
	Llm_Node_1(IdentifyInactiveCustomers) --> Database_Node[/"Query CRM Database(DatabaseNode)"/]
	Database_Node[/"Query CRM Database(DatabaseNode)"/] --> Llm_Node_2(GenerateEmailContent)
	Llm_Node_2(GenerateEmailContent) --> Llm_Node_3(ReviewEmailContent)
	Llm_Node_3(ReviewEmailContent) --> Llm_Node_4(CreateEmailLogic)
	Llm_Node_4(CreateEmailLogic) --> Api_Node[/"Send Emails(APINode)"/]
	Api_Node[/"Send Emails(APINode)"/] --> Llm_Node_5(MonitorCampaign)
	Llm_Node_5(MonitorCampaign) --> Llm_Node_6(GenerateFollowUpStrategy)
	Llm_Node_6(GenerateFollowUpStrategy) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to run a mailing campaign targeting customers who have not made a purchase in the last 6 months. Each node in this graph has a specific role. You will analyze data, generate content, review outcomes, and strategize follow-ups while collaborating with other nodes. Ensure that your responses are clear, actionable, and conducive to the next steps in the process. The goal is to effectively engage inactive customers and encourage them to make a purchase through personalized email communications.
##### IdentifyInactiveCustomers
Analyze the CRM data provided and identify customers who have not made any purchases in the last 6 months. Provide a detailed list of these customers, including their names, email addresses, and last purchase dates. Ensure the data is structured in a way that it can be easily processed by the subsequent nodes.
##### GenerateEmailContent
Using the list of inactive customers received from the previous node, generate personalized email content for each customer. The email must include a warm greeting, a reminder of their previous interactions, a unique discount code for their next purchase, a link to the website, and a compelling call to action encouraging them to make a purchase. Ensure that the content is engaging and tailored to resonate with each customer based on their past behavior.
##### ReviewEmailContent
Review the personalized email content generated for the inactive customers. Evaluate the effectiveness of the messaging, clarity, and appeal. Provide feedback on any areas that need improvement, such as tone, clarity, or additional elements that could enhance engagement. If necessary, suggest modifications to ensure the email content is compelling and aligns with best practices for customer re-engagement.
##### CreateEmailLogic
Design the logic required to send the personalized emails to the identified inactive customers. Outline the sequence of actions, including how to handle errors, track sent emails, and ensure that each customer receives their unique discount code. Additionally, provide a mechanism for confirming successful delivery and capturing engagement metrics such as open rates and click-through rates.
##### MonitorCampaign
After the emails have been sent, analyze the responses and effectiveness of the campaign. Gather data on open rates, click-through rates, and conversion rates. Provide a summary of the campaign's performance, highlighting any trends or insights. If specific customers re-engaged, note their responses and any patterns that emerge from the data collected.
##### GenerateFollowUpStrategy
Based on the effectiveness data from the previous node, generate a follow-up strategy for the next steps. Consider how to further engage customers who opened the email but did not make a purchase, as well as strategies for those who did convert. Suggest additional touchpoints, such as reminders or personalized offers, and outline a timeline for executing these follow-ups.

# Agent ID: agent-0c7c3096-5ecc-4d53-8e4c-2bd99e3fe4e2
## Fitness: 0.6816730792171299
```mermaid 
graph LR
	Start_Node[start_node] --> 1(CustomerSegmentationLLM)
	1(CustomerSegmentationLLM) --> 2[/"FetchCustomerData(DatabaseNode)"/]
	2[/"FetchCustomerData(DatabaseNode)"/] --> 3(EmailPersonalizationLLM)
	3(EmailPersonalizationLLM) --> 4[/"GenerateDiscountCode(PythonNode)"/]
	4[/"GenerateDiscountCode(PythonNode)"/] --> 5[/"SendEmails(APINode)"/]
	5[/"SendEmails(APINode)"/] --> 6(CampaignStrategyReviewLLM)
	6(CampaignStrategyReviewLLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to run a mailing campaign targeting customers who have not made a purchase in the last six months. Each node in this graph has a specific role in the problem-solving process. Your task is to utilize your reasoning and analytical capabilities to fulfill your designated function. You will exchange information with other nodes in the system to ensure a cohesive workflow. Be sure to provide clear and actionable insights that will be relayed to the next node in the process.
##### CustomerSegmentationLLM
Analyze the customer database and identify all customers who have not made a purchase in the last six months. Provide a detailed list of these customers, including their contact information and any relevant purchase history. Ensure your output is structured and easy to interpret, as it will be used for the next node in the graph.
##### EmailPersonalizationLLM
Using the list of customers who have not purchased in the last six months, create personalized email content for each customer. The email should include a friendly greeting, mention their past interactions with our brand, include a unique discount code for their next purchase, and provide a link to our website. Ensure the tone is engaging and encourages action. Format your output in a way that can be easily parsed for sending.
##### CampaignStrategyReviewLLM
After the mailing campaign has been executed, evaluate the overall strategy used in this campaign. Reflect on the effectiveness of the segmentation, personalization, and messaging. Provide constructive feedback on what worked well, areas for improvement, and suggestions for future campaigns. Your insights should be clear and actionable, aiding in the refinement of our marketing approach.



2025-04-08 16:49:12.284 | DEBUG    | ebiose.core.evo_forging_cycle:save_current_state:83 - Saving current state to data/2025-04-08_14-46-11/generation=0
2025-04-08 16:49:12.286 | INFO     | ebiose.core.evo_forging_cycle:run_generation:263 - Starting crossover and mutation...
2025-04-08 16:50:33.346 | DEBUG    | ebiose.core.engines.graph_engine.utils:get_structured_output_agent:50 - 
Initializing structured output agent for model AgentOutput (2)
2025-04-08 16:51:17.598 | DEBUG    | ebiose.core.evo_forging_cycle:crossover_and_mutate:391 - Number of offsprings: 1/1
2025-04-08 16:51:17.602 | INFO     | ebiose.core.evo_forging_cycle:run_generation:267 - Crossover and mutation completed in 0:02:05.314854 for a total cost of 0.0030828599999999998 $
2025-04-08 16:51:17.602 | INFO     | ebiose.core.evo_forging_cycle:run_generation:275 - Generation 0 completed in 0:02:05.321594 with a total cost of 0.0030828599999999998 $
2025-04-08 16:51:17.603 | INFO     | ebiose.core.evo_forging_cycle:execute

# Agent ID: agent-0c7c3096-5ecc-4d53-8e4c-2bd99e3fe4e2
## Fitness: 0.9549230540872181
```mermaid 
graph LR
	Start_Node[start_node] --> 1(CustomerSegmentationLLM)
	1(CustomerSegmentationLLM) --> 2[/"FetchCustomerData(DatabaseNode)"/]
	2[/"FetchCustomerData(DatabaseNode)"/] --> 3(EmailPersonalizationLLM)
	3(EmailPersonalizationLLM) --> 4[/"GenerateDiscountCode(PythonNode)"/]
	4[/"GenerateDiscountCode(PythonNode)"/] --> 5[/"SendEmails(APINode)"/]
	5[/"SendEmails(APINode)"/] --> 6(CampaignStrategyReviewLLM)
	6(CampaignStrategyReviewLLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to run a mailing campaign targeting customers who have not made a purchase in the last six months. Each node in this graph has a specific role in the problem-solving process. Your task is to utilize your reasoning and analytical capabilities to fulfill your designated function. You will exchange information with other nodes in the system to ensure a cohesive workflow. Be sure to provide clear and actionable insights that will be relayed to the next node in the process.
##### CustomerSegmentationLLM
Analyze the customer database and identify all customers who have not made a purchase in the last six months. Provide a detailed list of these customers, including their contact information and any relevant purchase history. Ensure your output is structured and easy to interpret, as it will be used for the next node in the graph.
##### EmailPersonalizationLLM
Using the list of customers who have not purchased in the last six months, create personalized email content for each customer. The email should include a friendly greeting, mention their past interactions with our brand, include a unique discount code for their next purchase, and provide a link to our website. Ensure the tone is engaging and encourages action. Format your output in a way that can be easily parsed for sending.
##### CampaignStrategyReviewLLM
After the mailing campaign has been executed, evaluate the overall strategy used in this campaign. Reflect on the effectiveness of the segmentation, personalization, and messaging. Provide constructive feedback on what worked well, areas for improvement, and suggestions for future campaigns. Your insights should be clear and actionable, aiding in the refinement of our marketing approach.

# Agent ID: agent-0ffe078b-5c90-4f39-83a9-ceef8dcd48f5
## Fitness: 0.901238719844738
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(IdentifyInactiveCustomers)
	Llm_Node_1(IdentifyInactiveCustomers) --> Database_Node[/"Query CRM Database(DatabaseNode)"/]
	Database_Node[/"Query CRM Database(DatabaseNode)"/] --> Llm_Node_2(GenerateEmailContent)
	Llm_Node_2(GenerateEmailContent) --> Llm_Node_3(ReviewEmailContent)
	Llm_Node_3(ReviewEmailContent) --> Llm_Node_4(CreateEmailLogic)
	Llm_Node_4(CreateEmailLogic) --> Llm_Node_5(SendEmails)
	Llm_Node_5(SendEmails) --> Api_Node[/"Send Emails(APINode)"/]
	Api_Node[/"Send Emails(APINode)"/] --> Llm_Node_6(MonitorCampaign)
	Llm_Node_6(MonitorCampaign) --> Llm_Node_7(GenerateFollowUpStrategy)
	Llm_Node_7(GenerateFollowUpStrategy) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to run a mailing campaign targeting customers who have not made a purchase in the last 6 months. Each node in this graph has a specific role. You will analyze data, generate content, review outcomes, and strategize follow-ups while collaborating with other nodes. Ensure that your responses are clear, actionable, and conducive to the next steps in the process. The goal is to effectively engage inactive customers and encourage them to make a purchase through personalized email communications.
##### IdentifyInactiveCustomers
Analyze the CRM data provided and identify customers who have not made any purchases in the last 6 months. Provide a detailed list of these customers, including their names, email addresses, and last purchase dates. Ensure the data is structured in a way that it can be easily processed by the subsequent nodes.
##### GenerateEmailContent
Using the list of inactive customers received from the previous node, generate personalized email content for each customer. The email must include a warm greeting, a reminder of their previous interactions, a unique discount code for their next purchase, a link to the website, and a compelling call to action encouraging them to make a purchase. Ensure that the content is engaging and tailored to resonate with each customer based on their past behavior.
##### ReviewEmailContent
Review the personalized email content generated for the inactive customers. Evaluate the effectiveness of the messaging, clarity, and appeal. Provide feedback on any areas that need improvement, such as tone, clarity, or additional elements that could enhance engagement. If necessary, suggest modifications to ensure the email content is compelling and aligns with best practices for customer re-engagement.
##### CreateEmailLogic
Design the logic required to send the personalized emails to the identified inactive customers. Outline the sequence of actions, including how to handle errors, track sent emails, and ensure that each customer receives their unique discount code. Additionally, provide a mechanism for confirming successful delivery and capturing engagement metrics such as open rates and click-through rates.
##### SendEmails
Using the logic designed in the previous node, trigger the API to send the emails to the identified inactive customers.
##### MonitorCampaign
After the emails have been sent, analyze the responses and effectiveness of the campaign. Gather data on open rates, click-through rates, and conversion rates. Provide a summary of the campaign's performance, highlighting any trends or insights. If specific customers re-engaged, note their responses and any patterns that emerge from the data collected.
##### GenerateFollowUpStrategy
Based on the effectiveness data from the previous node, generate a follow-up strategy for the next steps. Consider how to further engage customers who opened the email but did not make a purchase, as well as strategies for those who did convert. Suggest additional touchpoints, such as reminders or personalized offers, and outline a timeline for executing these follow-ups.



2025-04-08 16:51:17.617 | DEBUG    | ebiose.core.evo_forging_cycle:save_current_state:83 - Saving current state to data/2025-04-08_14-46-11/generation=1
2025-04-08 16:51:17.698 | INFO     | ebiose.core.evo_forging_cycle:run_generation:263 - Starting crossover and mutation...
2025-04-08 16:53:12.090 | DEBUG    | ebiose.core.evo_forging_cycle:crossover_and_mutate:391 - Number of offsprings: 1/1
2025-04-08 16:53:12.091 | INFO     | ebiose.core.evo_forging_cycle:run_generation:267 - Crossover and mutation completed in 0:01:54.392225 for a total cost of 0.002139555 $
2025-04-08 16:53:12.091 | INFO     | ebiose.core.evo_forging_cycle:run_generation:275 - Generation 1 completed in 0:01:54.488244 with a total cost of 0.002139555 $
2025-04-08 16:53:12.092 | INFO     | ebiose.core.evo_forging_cycle:execute_a_cycle:209 - Budget left after new generation: 0.483008135 $
2025-04-08 16:53:12.092 | INFO     | ebiose.core.evo_forging_cycle:run_generation:245 - ****** Running generation 2 ******
2025-04

# Agent ID: agent-bd7231e0-8021-4cfe-ba60-42f5c5f04cf9
## Fitness: 0.23244141958534903
```mermaid 
graph LR
	Start_Node[start_node] --> 1(CustomerSegmentationLLM)
	1(CustomerSegmentationLLM) --> 2[/"FetchCustomerData(DatabaseNode)"/]
	2[/"FetchCustomerData(DatabaseNode)"/] --> 3(EmailPersonalizationLLM)
	3(EmailPersonalizationLLM) --> 4[/"GenerateDiscountCode(PythonNode)"/]
	4[/"GenerateDiscountCode(PythonNode)"/] --> 5[/"SendEmails(APINode)"/]
	5[/"SendEmails(APINode)"/] --> 6(CampaignStrategyReviewLLM)
	6(CampaignStrategyReviewLLM) --> 7[/"FeedbackLoopNode(UserQueryNode)"/]
	7[/"FeedbackLoopNode(UserQueryNode)"/] --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to run a mailing campaign targeting customers who have not made a purchase in the last six months. Each node in this graph has a specific role in the problem-solving process. Your task is to utilize your reasoning and analytical capabilities to fulfill your designated function. You will exchange information with other nodes in the system to ensure a cohesive workflow. Be sure to provide clear and actionable insights that will be relayed to the next node in the process.
##### CustomerSegmentationLLM
Analyze the customer database and identify all customers who have not made a purchase in the last six months. Provide a detailed list of these customers, including their contact information and any relevant purchase history. Ensure your output is structured and easy to interpret, as it will be used for the next node in the graph.
##### EmailPersonalizationLLM
Using the list of customers who have not purchased in the last six months, craft personalized email content for each customer. The email should include a friendly greeting, reference their past interactions, include a unique discount code for their next purchase, and provide a link to our website. Ensure the tone is engaging and encourages action. Format your output for easy parsing by the email sending system.
##### CampaignStrategyReviewLLM
After the mailing campaign has been executed, evaluate the overall strategy used in this campaign. Reflect on the effectiveness of the segmentation, personalization, and messaging. Provide constructive feedback on what worked well, areas for improvement, and suggestions for future campaigns. Your insights should be clear and actionable, aiding in the refinement of our marketing approach.

# Agent ID: agent-0ffe078b-5c90-4f39-83a9-ceef8dcd48f5
## Fitness: 0.20104938602105327
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(IdentifyInactiveCustomers)
	Llm_Node_1(IdentifyInactiveCustomers) --> Database_Node[/"Query CRM Database(DatabaseNode)"/]
	Database_Node[/"Query CRM Database(DatabaseNode)"/] --> Llm_Node_2(GenerateEmailContent)
	Llm_Node_2(GenerateEmailContent) --> Llm_Node_3(ReviewEmailContent)
	Llm_Node_3(ReviewEmailContent) --> Llm_Node_4(CreateEmailLogic)
	Llm_Node_4(CreateEmailLogic) --> Llm_Node_5(SendEmails)
	Llm_Node_5(SendEmails) --> Api_Node[/"Send Emails(APINode)"/]
	Api_Node[/"Send Emails(APINode)"/] --> Llm_Node_6(MonitorCampaign)
	Llm_Node_6(MonitorCampaign) --> Llm_Node_7(GenerateFollowUpStrategy)
	Llm_Node_7(GenerateFollowUpStrategy) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to run a mailing campaign targeting customers who have not made a purchase in the last 6 months. Each node in this graph has a specific role. You will analyze data, generate content, review outcomes, and strategize follow-ups while collaborating with other nodes. Ensure that your responses are clear, actionable, and conducive to the next steps in the process. The goal is to effectively engage inactive customers and encourage them to make a purchase through personalized email communications.
##### IdentifyInactiveCustomers
Analyze the CRM data provided and identify customers who have not made any purchases in the last 6 months. Provide a detailed list of these customers, including their names, email addresses, and last purchase dates. Ensure the data is structured in a way that it can be easily processed by the subsequent nodes.
##### GenerateEmailContent
Using the list of inactive customers received from the previous node, generate personalized email content for each customer. The email must include a warm greeting, a reminder of their previous interactions, a unique discount code for their next purchase, a link to the website, and a compelling call to action encouraging them to make a purchase. Ensure that the content is engaging and tailored to resonate with each customer based on their past behavior.
##### ReviewEmailContent
Review the personalized email content generated for the inactive customers. Evaluate the effectiveness of the messaging, clarity, and appeal. Provide feedback on any areas that need improvement, such as tone, clarity, or additional elements that could enhance engagement. If necessary, suggest modifications to ensure the email content is compelling and aligns with best practices for customer re-engagement.
##### CreateEmailLogic
Design the logic required to send the personalized emails to the identified inactive customers. Outline the sequence of actions, including how to handle errors, track sent emails, and ensure that each customer receives their unique discount code. Additionally, provide a mechanism for confirming successful delivery and capturing engagement metrics such as open rates and click-through rates.
##### SendEmails
Using the logic designed in the previous node, trigger the API to send the emails to the identified inactive customers.
##### MonitorCampaign
After the emails have been sent, analyze the responses and effectiveness of the campaign. Gather data on open rates, click-through rates, and conversion rates. Provide a summary of the campaign's performance, highlighting any trends or insights. If specific customers re-engaged, note their responses and any patterns that emerge from the data collected.
##### GenerateFollowUpStrategy
Based on the effectiveness data from the previous node, generate a follow-up strategy for the next steps. Consider how to further engage customers who opened the email but did not make a purchase, as well as strategies for those who did convert. Suggest additional touchpoints, such as reminders or personalized offers, and outline a timeline for executing these follow-ups.



2025-04-08 16:53:12.095 | DEBUG    | ebiose.core.evo_forging_cycle:save_current_state:83 - Saving current state to data/2025-04-08_14-46-11/generation=2
2025-04-08 16:53:12.096 | INFO     | ebiose.core.evo_forging_cycle:run_generation:263 - Starting crossover and mutation...
2025-04-08 16:54:09.084 | DEBUG    | ebiose.core.evo_forging_cycle:crossover_and_mutate:391 - Number of offsprings: 1/1
2025-04-08 16:54:09.088 | INFO     | ebiose.core.evo_forging_cycle:run_generation:267 - Crossover and mutation completed in 0:00:56.991934 for a total cost of 0.002389365 $
2025-04-08 16:54:09.089 | INFO     | ebiose.core.evo_forging_cycle:run_generation:275 - Generation 2 completed in 0:00:56.997322 with a total cost of 0.002389365 $
2025-04-08 16:54:09.090 | INFO     | ebiose.core.evo_forging_cycle:execute_a_cycle:209 - Budget left after new generation: 0.48061877 $
2025-04-08 16:54:09.090 | INFO     | ebiose.core.evo_forging_cycle:run_generation:245 - ****** Running generation 3 ******
2025-04-

# Agent ID: agent-0ffe078b-5c90-4f39-83a9-ceef8dcd48f5
## Fitness: 0.7845056935353862
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(IdentifyInactiveCustomers)
	Llm_Node_1(IdentifyInactiveCustomers) --> Database_Node[/"Query CRM Database(DatabaseNode)"/]
	Database_Node[/"Query CRM Database(DatabaseNode)"/] --> Llm_Node_2(GenerateEmailContent)
	Llm_Node_2(GenerateEmailContent) --> Llm_Node_3(ReviewEmailContent)
	Llm_Node_3(ReviewEmailContent) --> Llm_Node_4(CreateEmailLogic)
	Llm_Node_4(CreateEmailLogic) --> Llm_Node_5(SendEmails)
	Llm_Node_5(SendEmails) --> Api_Node[/"Send Emails(APINode)"/]
	Api_Node[/"Send Emails(APINode)"/] --> Llm_Node_6(MonitorCampaign)
	Llm_Node_6(MonitorCampaign) --> Llm_Node_7(GenerateFollowUpStrategy)
	Llm_Node_7(GenerateFollowUpStrategy) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to run a mailing campaign targeting customers who have not made a purchase in the last 6 months. Each node in this graph has a specific role. You will analyze data, generate content, review outcomes, and strategize follow-ups while collaborating with other nodes. Ensure that your responses are clear, actionable, and conducive to the next steps in the process. The goal is to effectively engage inactive customers and encourage them to make a purchase through personalized email communications.
##### IdentifyInactiveCustomers
Analyze the CRM data provided and identify customers who have not made any purchases in the last 6 months. Provide a detailed list of these customers, including their names, email addresses, and last purchase dates. Ensure the data is structured in a way that it can be easily processed by the subsequent nodes.
##### GenerateEmailContent
Using the list of inactive customers received from the previous node, generate personalized email content for each customer. The email must include a warm greeting, a reminder of their previous interactions, a unique discount code for their next purchase, a link to the website, and a compelling call to action encouraging them to make a purchase. Ensure that the content is engaging and tailored to resonate with each customer based on their past behavior.
##### ReviewEmailContent
Review the personalized email content generated for the inactive customers. Evaluate the effectiveness of the messaging, clarity, and appeal. Provide feedback on any areas that need improvement, such as tone, clarity, or additional elements that could enhance engagement. If necessary, suggest modifications to ensure the email content is compelling and aligns with best practices for customer re-engagement.
##### CreateEmailLogic
Design the logic required to send the personalized emails to the identified inactive customers. Outline the sequence of actions, including how to handle errors, track sent emails, and ensure that each customer receives their unique discount code. Additionally, provide a mechanism for confirming successful delivery and capturing engagement metrics such as open rates and click-through rates.
##### SendEmails
Using the logic designed in the previous node, trigger the API to send the emails to the identified inactive customers.
##### MonitorCampaign
After the emails have been sent, analyze the responses and effectiveness of the campaign. Gather data on open rates, click-through rates, and conversion rates. Provide a summary of the campaign's performance, highlighting any trends or insights. If specific customers re-engaged, note their responses and any patterns that emerge from the data collected.
##### GenerateFollowUpStrategy
Based on the effectiveness data from the previous node, generate a follow-up strategy for the next steps. Consider how to further engage customers who opened the email but did not make a purchase, as well as strategies for those who did convert. Suggest additional touchpoints, such as reminders or personalized offers, and outline a timeline for executing these follow-ups.

# Agent ID: agent-4f671399-33ad-4bcd-8ec7-4800d1d06be4
## Fitness: 0.4724345997074011
```mermaid 
graph LR
	Start_Node[start_node] --> 1(CustomerSegmentationLLM)
	1(CustomerSegmentationLLM) --> 2[/"FetchCustomerData(DatabaseNode)"/]
	2[/"FetchCustomerData(DatabaseNode)"/] --> 3(EmailPersonalizationLLM)
	3(EmailPersonalizationLLM) --> 4[/"GenerateDiscountCode(PythonNode)"/]
	4[/"GenerateDiscountCode(PythonNode)"/] --> 5[/"SendEmails(APINode)"/]
	5[/"SendEmails(APINode)"/] --> 6(CampaignStrategyReviewLLM)
	6(CampaignStrategyReviewLLM) --> 7[/"FeedbackLoopNode(UserQueryNode)"/]
	7[/"FeedbackLoopNode(UserQueryNode)"/] --> 8(CustomerEngagementAnalysisLLM)
	8(CustomerEngagementAnalysisLLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to run a mailing campaign targeting customers who have not made a purchase in the last six months. Each node in this graph has a specific role in the problem-solving process. Your task is to utilize your reasoning and analytical capabilities to fulfill your designated function. You will exchange information with other nodes in the system to ensure a cohesive workflow. Be sure to provide clear and actionable insights that will be relayed to the next node in the process.
##### CustomerSegmentationLLM
Analyze the customer database and identify all customers who have not made a purchase in the last six months. Provide a detailed list of these customers, including their contact information and any relevant purchase history. Ensure your output is structured and easy to interpret, as it will be used for the next node in the graph.
##### EmailPersonalizationLLM
Using the list of customers who have not purchased in the last six months, craft personalized email content for each customer. The email should include a friendly greeting, reference their past interactions, include a unique discount code for their next purchase, and provide a link to our website. Ensure the tone is engaging and encourages action. Format your output for easy parsing by the email sending system.
##### CampaignStrategyReviewLLM
After the mailing campaign has been executed, evaluate the overall strategy used in this campaign. Reflect on the effectiveness of the segmentation, personalization, and messaging. Provide constructive feedback on what worked well, areas for improvement, and suggestions for future campaigns. Your insights should be clear and actionable, aiding in the refinement of our marketing approach.
##### CustomerEngagementAnalysisLLM
Evaluate the customer engagement metrics after the email campaign. Identify trends in customer responses, purchase behaviors, and overall campaign effectiveness. Provide insights that can guide future marketing strategies.



2025-04-08 16:54:09.097 | DEBUG    | ebiose.core.evo_forging_cycle:save_current_state:83 - Saving current state to data/2025-04-08_14-46-11/generation=3
2025-04-08 16:54:09.099 | INFO     | ebiose.core.evo_forging_cycle:run_generation:263 - Starting crossover and mutation...


KeyboardInterrupt: 

In [ ]:
logger.info("Best agents and their fitness:")
forge.display_results(best_agents, best_finess)